In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/codebase/codebase.txt


In [8]:
!pip install -q scikit-learn transformers

^C
ERROR: Operation cancelled by user


In [ ]:

# 1. Import Libraries
import os
import re
import torch
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_name = "tiiuae/falcon-7b-instruct"
# query_wrapper_prompt = "Context: {context}\n\nQuestion: {query}\n\nAnswer:"

def load_file(file_path):
    """
    Reads the content of a text file.
    """
    with open(file_path, 'r') as file:
        content = file.read()
    return content


# Improved TF-IDF Indexing
def create_tfidf_index(segments):
    """
    Converts text segments into TF-IDF vectors with n-gram support for better matching.
    """
    vectorizer = TfidfVectorizer(
        stop_words='english',
        ngram_range=(1, 3),  # Unigrams, bigrams, and trigrams
        max_features=10000
    )
    segment_vectors = vectorizer.fit_transform(segments)
    return vectorizer, segment_vectors

# 4. Query Processing and Retrieval
def retrieve_relevant_segments(query, vectorizer, segment_vectors, segments, top_k=3):
    """
    Finds the most relevant text segments based on cosine similarity with the query.
    """
    query_vector = vectorizer.transform([query])
    similarities = cosine_similarity(query_vector, segment_vectors).flatten()
    top_indices = similarities.argsort()[-top_k:][::-1]  # Top-k most relevant indices
    return [(segments[i], similarities[i]) for i in top_indices]

# Updated Preprocessing with Filtering
def preprocess_text(content):
    """
    Splits raw text into segments, filters out irrelevant ones, and retains meaningful text.
    """
    segments = re.split(r'\n\s*\n', content)  # Split by double newlines
    # Filter segments for relevance (e.g., no CSS or irrelevant snippets)
    filtered_segments = [
        segment.strip() for segment in segments
        if len(segment.split()) > 5 and not re.match(r'^[#.@]', segment.strip())
    ]
    return filtered_segments

llm = pipeline(
    "text-generation",
    model=AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16,
        device_map="auto",  # Automatically distribute layers across GPUs
        trust_remote_code=True,  # Required for Falcon's custom code
        # quantization_config=quantization_config,  # Optional for memory optimization
    ),
    tokenizer=AutoTokenizer.from_pretrained(model_name),
    max_new_tokens=256,  # Maximum number of tokens to generate
    # context_window=4096,  # Set context window size
    # generate_kwargs={"do_sample": False},
)


In [7]:
import gc

gc.collect()

7738

In [8]:

# Updated Answer Generation Function
def generate_answer(query, retrieved_segments):
    """
    Generates an answer using Falcon-7B with multiple GPUs.
    """
    # Combine retrieved segments into context
    context = " ".join([segment for segment, _ in retrieved_segments])
    prompt = (
        f"You are an intelligent assistant. Read the context below and answer the question specifically.\n\n"
        f"Context: {context}\n\n"
        f"Question: {query}\n"
        f"Provide the name of the library used for infinite scrolling in the application.\n\n"
        f"Answer:"
    )
    
    # Generate answer
    response = llm(
        prompt,
        max_new_tokens=50,  # Generate up to 50 new tokens
        num_return_sequences=1,
    )
    return response[0]['generated_text']


# Usage Example
# Update the file path to your uploaded file
file_path = "/kaggle/input/codebase/codebase.txt"  # Replace with your file path
query = "Which library is used for infinite scrolling?"


# Load and preprocess the file
raw_text = load_file(file_path)
segments = preprocess_text(raw_text)

# Index segments with TF-IDF
vectorizer, segment_vectors = create_tfidf_index(segments)

# Retrieve relevant segments
retrieved_segments = retrieve_relevant_segments(query, vectorizer, segment_vectors, segments, top_k=5)

# Print retrieved segments
print("Retrieved Segments:")
for i, (segment, score) in enumerate(retrieved_segments):
    print(f"\nSegment {i+1} (Score: {score:.4f}):\n{segment}")
    print("-" * 50)

# Generate and return an answer
print("\nGenerating Answer...\n")
answer = generate_answer(query, retrieved_segments)
print(answer)


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Retrieved Segments:

Segment 1 (Score: 0.2793):
if(!page_id || (page_id < 0) ){
             localStorage.setItem("page","1");
        }
        url = `/?page=${page_id}`;
        if ((window.innerHeight + window.scrollY) >= document.body.scrollHeight - 100) {
        document.getElementById("infinite-scrolling").classList.add("show")
            let is_loading = localStorage.getItem("loading");
            if(is_loading == "false"){
                localStorage.setItem("loading","true");
                $.ajax({
                    url: url,
                    method: "get",
                    }).then((res) => {
                            $('.postsContainer').append(res);
                            document.getElementById("infinite-scrolling").classList.remove("show")
                            const post_contents = [...$(".post-content")]
                            post_contents.forEach(post => {
                                const classList = post.classList
                 